In [ ]:
!pip -q install requests pandas tqdm


In [ ]:
!pip -q install cloudscraper beautifulsoup4


In [ ]:
COUNTRIES = ["US","CA","BR","AR","TR","PL","DE","FR","GB","JP","AU","CN"]
LANG = "english"
TOPN = 50          #  first 50


In [ ]:
import os, time, csv, pathlib, re
from typing import List, Dict, Optional
import requests, pandas as pd
from tqdm import tqdm

def ensure_outdir(d: str) -> str:
    pathlib.Path(d).mkdir(parents=True, exist_ok=True)
    return d

def write_csv(rows: List[Dict], out_path: str):
    if not rows:
        print(f"[WARN] No rows -> {out_path}")
        return
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
        w.writeheader(); w.writerows(rows)
    print(f"[OK] {len(rows)} rows -> {out_path}")

def backoff_sleep(attempt: int, base=0.6, cap=8.0):
    time.sleep(min(cap, base * (2 ** attempt)))


In [ ]:
import re, time, requests, pandas as pd
from bs4 import BeautifulSoup
import cloudscraper

def fetch_steam_topsellers(cc="US", lang="english", topn=50, verbose=True):

    ua = ("Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
          "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 topseller-bot")
    headers_json = {
        "User-Agent": ua,
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "en-US,en;q=0.9"
    }
    headers_html = {
        "User-Agent": ua,
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://store.steampowered.com/"
    }

    urlA = f"https://store.steampowered.com/api/featuredcategories/?cc={cc}&l={lang}"
    for attempt in range(2):
        try:
            r = requests.get(urlA, headers=headers_json, timeout=20)
            if r.status_code >= 500: raise requests.HTTPError(f"HTTP {r.status_code}")
            r.raise_for_status()
            j = r.json() or {}
            items = (j.get("topsellers", {}) or {}).get("items", [])
            if not items:
                items = (j.get("featured_categories", {}).get("topsellers", {}) or {}).get("items", [])
            if items:
                rows = []
                for it in items[:topn]:
                    rows.append({
                        "appid": it.get("id"),
                        "name": it.get("name"),
                        "cc": cc,
                        "currency": it.get("currency"),
                        "initial": (it.get("original_price") or 0)/100 if isinstance(it.get("original_price"), (int,float)) else None,
                        "final": (it.get("final_price") or 0)/100 if isinstance(it.get("final_price"), (int,float)) else None,
                        "discount_percent": it.get("discount_percent"),
                    })
                if verbose: print(f"[A] got {len(rows)} from featuredcategories")
                return rows
        except Exception as e:
            if verbose: print(f"[A attempt {attempt}] {e}")
            time.sleep(0.8 * (2**attempt))

    urlB = (f"https://store.steampowered.com/search/results/"
            f"?query=&start=0&count={topn}&filter=topsellers&cc={cc}&l={lang}&v=4")
    for attempt in range(2):
        try:
            r = requests.get(urlB, headers=headers_json, timeout=20)
            if r.status_code >= 500: raise requests.HTTPError(f"HTTP {r.status_code}")
            r.raise_for_status()
            j = r.json()
            html = j.get("results_html", "")
            appids = re.findall(r'data-ds-appid="(\d+)"', html)
            names  = re.findall(r'<span class="title">([^<]+)</span>', html)
            rows = []
            for i, appid in enumerate(appids[:topn]):
                name = names[i] if i < len(names) else None
                rows.append({"appid": appid, "name": name, "cc": cc,
                             "currency": None, "initial": None, "final": None, "discount_percent": None})
            if rows:
                if verbose: print(f"[B] got {len(rows)} from search v=4")
                return rows
        except Exception as e:
            if verbose: print(f"[B attempt {attempt}] {e}")
            time.sleep(0.8 * (2**attempt))

    urlC = f"https://store.steampowered.com/search/?filter=topsellers&cc={cc}&l={lang}"
    scraper = cloudscraper.create_scraper(browser={'custom': ua})
    for attempt in range(3):
        try:
            r = scraper.get(urlC, headers=headers_html, timeout=30)
            if r.status_code >= 500: raise requests.HTTPError(f"HTTP {r.status_code}")
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")
            rows = []
            for a in soup.select("a.search_result_row"):
                appid = a.get("data-ds-appid")
                if not appid:
                    continue
                name_tag = a.select_one("span.title")
                name = name_tag.get_text(strip=True) if name_tag else None
                rows.append({"appid": appid, "name": name, "cc": cc,
                             "currency": None, "initial": None, "final": None, "discount_percent": None})
                if len(rows) >= topn:
                    break
            if rows:
                if verbose: print(f"[C] got {len(rows)} from HTML search")
                return rows
        except Exception as e:
            if verbose: print(f"[C attempt {attempt}] {e}")
            time.sleep(1.2 * (2**attempt))

    return []


topsellers_rows = fetch_steam_topsellers(cc="US", lang="english", topn=50)
write_csv(topsellers_rows, "steam_topsellers.csv")
pd.DataFrame(topsellers_rows).head()


[B attempt 0] Expecting value: line 2 column 1 (char 1)
[B attempt 1] Expecting value: line 2 column 1 (char 1)
[C] got 50 from HTML search
[OK] 50 rows -> steam_topsellers.csv


,appid,name,cc,currency,initial,final,discount_percent
0,1675200,Steam Deck,US,None,None,None,None
1,1004640,FINAL FANTASY TACTICS - The Ivalice Chronicles,US,None,None,None,None
2,730,Counter-Strike 2,US,None,None,None,None
3,2807960,Battlefield™ 6,US,None,None,None,None
4,3405340,Megabonk,US,None,None,None,None


In [ ]:
!pip -q install cloudscraper

In [ ]:
import os, time, csv, random, json, pathlib
from typing import Optional, Dict, List, Tuple
import pandas as pd
import cloudscraper
from tqdm import tqdm

OUT_PRICES_CSV = "steam_prices.csv"
LANG = "english"


df_top = pd.read_csv("steam_topsellers.csv")

appids = [str(x) for x in pd.to_numeric(df_top["appid"], errors="coerce").dropna().astype(int).astype(str).unique()]


if os.path.exists(OUT_PRICES_CSV):
    df_done = pd.read_csv(OUT_PRICES_CSV)
    done_set: set[Tuple[str,str]] = set(zip(df_done["appid"].astype(str), df_done["country"].astype(str)))
    out_rows = df_done.to_dict("records")
    print(f"[RESUME] loaded {len(df_done)} rows from {OUT_PRICES_CSV}")
else:
    done_set = set()
    out_rows: List[Dict] = []

tasks: List[Tuple[str,str]] = []
for aid in appids:
    for cc in COUNTRIES:
        if (aid, cc) not in done_set:
            tasks.append((aid, cc))
random.shuffle(tasks)
print(f"[PLAN] to fetch {len(tasks)} appid×country pairs")

scraper = cloudscraper.create_scraper(browser={
    'browser': 'chrome',
    'platform': 'linux',
    'mobile': False
})
UA = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                    "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 pricebot/2.0",
      "Accept": "application/json, text/javascript, */*; q=0.01",
      "Accept-Language": "en-US,en;q=0.9"}

def fetch_steam_price_throttled(appid: str, country: str, lang: str = LANG) -> Optional[Dict]:
    """
    稳健请求 appdetails?appids=<APPID>&cc=<CC>&l=<LANG>
    - 429: 读取 Retry-After 或指数回退 + 随机抖动
    - 5xx/网络错误: 指数回退
    - 正常: 展平 price_overview 并返回 dict
    """
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc={country}&l={lang}"
    base_sleep = 1.0
    attempt = 0
    while attempt <= 8:
        try:
            r = scraper.get(url, headers=UA, timeout=30)

            if r.status_code == 429:
                retry_after = 0
                try:
                    retry_after = int(r.headers.get("Retry-After", "0"))
                except Exception:
                    pass

                sleep_s = retry_after if retry_after > 0 else (min(60, (2 ** attempt) * 2) + random.uniform(0, 2))
            elif r.status_code >= 500:

                sleep_s = min(45, (2 ** attempt) * 1.5 + random.uniform(0, 1.5))
            else:
                r.raise_for_status()
                j = r.json()
                data = j.get(str(appid), {}).get("data", {})
                pov  = data.get("price_overview")
                if not pov:
                    return None
                initial, final = pov.get("initial"), pov.get("final")
                return {
                    "platform": "steam",
                    "appid": str(appid),
                    "country": country,
                    "currency": pov.get("currency"),
                    "initial": (initial or 0)/100 if isinstance(initial, (int,float)) else None,
                    "final": (final or 0)/100 if isinstance(final, (int,float)) else None,
                    "discount_percent": pov.get("discount_percent"),
                    "release_date": (data.get("release_date") or {}).get("date"),
                }

            attempt += 1
            time.sleep(sleep_s)
            continue
        except Exception:
            attempt += 1
            time.sleep(min(45, (2 ** attempt) * 1.2 + random.uniform(0, 1.0)))

    return None


CHUNK_PAUSE = 10.0
CHUNK_EVERY = 80
SAVE_EVERY  = 50

processed = 0
for idx, (aid, cc) in enumerate(tqdm(tasks, desc="Steam regional prices"), start=1):
    rec = fetch_steam_price_throttled(aid, cc)
    if rec:
        out_rows.append(rec)
        done_set.add((aid, cc))
    processed += 1

    time.sleep(0.6 + random.uniform(0, 0.5))

    if idx % CHUNK_EVERY == 0:
        time.sleep(CHUNK_PAUSE)

    if (len(out_rows) % SAVE_EVERY == 0) and out_rows:
        tmp_path = OUT_PRICES_CSV + ".tmp"
        with open(tmp_path, "w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=list(out_rows[0].keys()))
            w.writeheader(); w.writerows(out_rows)
        os.replace(tmp_path, OUT_PRICES_CSV)

if out_rows:
    with open(OUT_PRICES_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=list(out_rows[0].keys()))
        w.writeheader(); w.writerows(out_rows)
print(f"[DONE] wrote {len(out_rows)} rows -> {OUT_PRICES_CSV}")


[PLAN] to fetch 600 appid×country pairs


Steam regional prices: 100%|██████████| 600/600 [12:34<00:00,  1.26s/it]

[DONE] wrote 513 rows -> steam_prices.csv


In [ ]:
def build_summary(price_csv="steam_prices.csv", topseller_csv="steam_topsellers.csv", out_path="steam_prices_summary.csv"):
    p = pd.read_csv(price_csv)           # appid,country,currency,final...
    t = pd.read_csv(topseller_csv)       # appid,name
    df = p.merge(t[["appid","name"]].drop_duplicates(), on="appid", how="left")
    pivot = df.pivot_table(index=["appid","name"], columns="country", values="final", aggfunc="first")
    pivot.reset_index(inplace=True)
    pivot.to_csv(out_path, index=False)
    print(f"[OK] summary -> {out_path}")
    return pivot

summary_df = build_summary()
summary_df.head()


[OK] summary -> steam_prices_summary.csv


country,appid,name,AR,AU,BR,CA,CN,DE,FR,GB,JP,PL,TR,US
0,252490,Rust,9.49,29.97,59.99,25.99,69.00,19.99,19.99,17.49,2300.0,59.99,9.49,19.99
1,275850,No Man's Sky,10.79,35.18,64.80,31.19,70.00,23.59,23.59,19.99,2600.0,109.99,10.79,23.99
2,281990,Stellaris,5.99,14.23,35.99,13.24,42.00,9.99,9.99,8.74,1347.0,46.24,5.99,9.99
3,294100,RimWorld,14.39,35.96,81.59,31.99,102.40,25.59,25.59,22.79,3120.0,107.99,14.39,27.99
4,311210,Call of Duty®: Black Ops III,19.79,31.66,108.86,26.39,111.54,19.79,19.79,14.84,2966.0,85.30,19.79,19.79


In [ ]:
import pandas as pd, os

def load_any(*candidates):
    for p in candidates:
        if p and os.path.exists(p):
            return pd.read_csv(p)
    raise FileNotFoundError(f"None found: {candidates}")

p = load_any("steam_prices.csv")
t = load_any("steam_topsellers.csv")


In [ ]:
import pandas as pd, time, datetime, requests
from tqdm import tqdm

LANG = "english"

def fetch_app_meta(appid, cc="US", lang=LANG, session=None):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc={cc}&l={lang}"
    s = session or requests
    for _ in range(3):
        try:
            r = s.get(url, timeout=20, headers={"User-Agent":"Mozilla/5.0 meta-bot/1.0"})
            r.raise_for_status()
            d = r.json().get(str(appid),{}).get("data",{})
            return {
                "appid": str(appid),
                "type": d.get("type"),
                "is_free": d.get("is_free"),
                "genres": ",".join([g.get("description") for g in (d.get("genres") or [])]),
                "categories": ",".join([g.get("description") for g in (d.get("categories") or [])]),
            }
        except Exception:
            time.sleep(0.7)
    return {"appid": str(appid), "type": None, "is_free": None, "genres": None, "categories": None}


p = pd.read_csv("steam_prices.csv")
t = pd.read_csv("steam_topsellers.csv")[["appid","name"]].drop_duplicates()
p["appid"] = p["appid"].astype(str); t["appid"] = t["appid"].astype(str)


appids = t["appid"].unique().tolist()
meta_rows = []
with requests.Session() as s:
    for a in tqdm(appids, desc="fetch meta"):
        meta_rows.append(fetch_app_meta(a, session=s))
meta = pd.DataFrame(meta_rows)


p2 = p.merge(meta, on="appid", how="left")
p2["ts_collected_utc"] = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


free_ids = set(meta.loc[meta["is_free"]==True,"appid"])
p2.loc[p2["appid"].isin(free_ids), ["final","initial"]] = 0.0

mask_game = (p2["type"].fillna("").str.lower()=="game")
p2 = p2[mask_game | p2["appid"].isin(free_ids)]

p2.to_csv("steam_prices_enriched.csv", index=False)
print("[OK] -> steam_prices_enriched.csv", len(p2))
display(p2.head())


fetch meta: 100%|██████████| 50/50 [00:06<00:00,  8.16it/s]

[OK] -> steam_prices_enriched.csv 492



/tmp/ipython-input-1052602436.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  p2["ts_collected_utc"] = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


,platform,appid,country,currency,initial,final,discount_percent,release_date,type,is_free,genres,categories,ts_collected_utc
0,steam,3164500,AR,USD,10.49,7.34,30,"24 Mar, 2025",game,False,"Action,Indie,Simulation,Strategy,Early Access","Single-player,Multi-player,Co-op,Online Co-op,...",2025-10-01T02:56:19Z
1,steam,1757300,AR,USD,7.99,7.99,0,"19 Sep, 2025",game,False,"Action,Adventure,Indie,Early Access","Single-player,Multi-player,Co-op,Online Co-op,...",2025-10-01T02:56:19Z
2,steam,1142710,JP,JPY,7000.00,1750.00,75,"16 Feb, 2022",game,False,"Action,Strategy","Single-player,Multi-player,PvP,Online PvP,LAN ...",2025-10-01T02:56:19Z
3,steam,1144200,GB,GBP,44.99,22.49,50,"13 Dec, 2023",game,False,"Action,Adventure,Indie","Single-player,Multi-player,Co-op,Online Co-op,...",2025-10-01T02:56:19Z
4,steam,1903340,BR,BRL,199.00,179.10,10,"24 Apr, 2025",game,False,"Action,RPG","Single-player,Steam Achievements,Full controll...",2025-10-01T02:56:19Z


In [ ]:
import pandas as pd

p2 = pd.read_csv("steam_prices_enriched.csv", dtype={"appid":str})
names = pd.read_csv("steam_topsellers.csv", dtype={"appid":str})[["appid","name"]].drop_duplicates()
df = p2.merge(names, on="appid", how="left")

pv = df.pivot_table(index=["appid","name"], columns="country", values="final", aggfunc="first").reset_index()

if "US" in pv.columns:
    for c in [col for col in pv.columns if col not in ["appid","name","US"]]:
        pv[f"diff_vs_US_{c}_%"] = (pv[c] - pv["US"]) / pv["US"] * 100.0

pv.to_csv("steam_prices_summary_diff.csv", index=False)
print("[OK] -> steam_prices_summary_diff.csv")
display(pv.head())


[OK] -> steam_prices_summary_diff.csv


country,appid,name,AR,AU,BR,CA,CN,DE,FR,GB,...,diff_vs_US_AU_%,diff_vs_US_BR_%,diff_vs_US_CA_%,diff_vs_US_CN_%,diff_vs_US_DE_%,diff_vs_US_FR_%,diff_vs_US_GB_%,diff_vs_US_JP_%,diff_vs_US_PL_%,diff_vs_US_TR_%
0,1004640,FINAL FANTASY TACTICS - The Ivalice Chronicles,39.99,74.95,219.90,66.99,298.0,49.99,49.99,39.99,...,49.929986,339.887978,34.006801,496.119224,0.000000,0.000000,-20.004001,11502.320464,298.079616,-20.004001
1,1030300,Hollow Knight: Silksong,6.99,29.50,59.99,25.99,76.0,19.50,19.50,16.75,...,47.573787,200.100050,30.015008,280.190095,-2.451226,-2.451226,-16.208104,11405.752876,275.137569,-60.030015
2,1086940,Baldur's Gate 3,26.24,67.46,149.99,59.99,223.5,44.99,44.99,37.49,...,49.944432,233.385197,33.340742,396.777062,0.000000,0.000000,-16.670371,14067.592798,315.092243,-41.675928
3,1091500,Cyberpunk 2077,15.74,31.48,69.96,27.99,104.3,20.99,20.99,17.49,...,49.976179,233.301572,33.349214,396.903287,0.000000,0.000000,-16.674607,14535.540734,231.824678,-25.011910
4,1142710,Total War: WARHAMMER III,12.49,24.98,74.75,19.99,74.5,14.99,14.99,12.49,...,66.644430,398.665777,33.355570,396.997999,0.000000,0.000000,-16.677785,11574.449633,381.987992,-16.677785


In [ ]:
from google.colab import files

files.download("steam_topsellers.csv")
files.download("steam_prices.csv")
files.download("steam_prices_enriched.csv")
files.download("steam_prices_summary_diff.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>